In [1]:
#!/usr/bin/env python

# Copyright (c) 2010- The University of Notre Dame.
# This software is distributed under the GNU General Public License.
# See the file COPYING for details.

import work_queue as wq
import ipywidgets as widgets
import time
from widget_control import get_user_input_widgets, get_output_widgets, create_progress_tracker

input_widgets = get_user_input_widgets()
display(widgets.HBox(input_widgets))

ImportError: cannot import name 'progress_tracker' from 'widget_control' (/afs/crc.nd.edu/user/d/dsimone2/Documents/interactive/work-queue-interactive/binder/widget_control.py)

In [ ]:
number_of_tasks = input_widgets[0]
number_of_workers = input_widgets[1]
starting_number = input_widgets[2]

def is_prime(number):
    import math, time
    for i in range(2, int(math.sqrt(number)) + 1):
            if (number % i == 0):
                return 0
    return 1

output_widgets = get_output_widgets(number_of_tasks.value, number_of_workers.value)

display(widgets.HBox([output_widgets[0], output_widgets[1], output_widgets[2]]))
display(output_widgets[3])
display(widgets.HBox(output_widgets[4]))

task_output_storage = []
create_progress_tracker(task_output_storage, number_of_tasks.value, starting_number.value, number_of_workers.value)

q = wq.WorkQueue(0)
workers = wq.Factory('local', manager_host_port=f'localhost:{q.port}')
workers.cores = 1
workers.memory = 100
workers.disk = 100
workers.min_workers = 5
workers.max_workers = number_of_workers.value

for i in range(starting_number.value, starting_number.value + number_of_tasks.value + 1):
    p_task = wq.PythonTask(is_prime, i)
    p_task.specify_environment("worker-env.tar.gz")
    q.submit(p_task)
    
starting_time = time.perf_counter()

with workers:
    while not q.empty():
        t = q.wait(1)
        if t:
            x = t.output
            if x == 1:
                task_output_storage[int(t.id) + 1].disabled = True
            task_output_storage[int(t.id) + 1].button_style = 'success'
            for i in range(q.stats.workers_connected + 1):
                if (int(t.id) + 1 - i) > 0:
                    if (task_output_storage[int(t.id) + 1 - i].button_style == ''):
                        task_output_storage[int(t.id) + 1 - i].button_style='warning'
        output_widgets[0].value = q.stats.tasks_done
        output_widgets[3].value = output_widgets[0].value / (time.perf_counter() - starting_time)
        output_widgets[2].value = q.stats.workers_idle
        output_widgets[1].value = q.stats.workers_connected
        #print(q.stats.workers_connected)
        #worker_array[0].value = str(q.workers_summary()[0]['workers'])
        #worker_array[1].value = str(q.workers_summary()[0]['cores'])
        #worker_array[2].value = str(q.workers_summary()[0]['memory'])
        #worker_array[3].value = str(q.workers_summary()[0]['disk'])
        #worker_array[4].value = str(q.workers_summary()[0]['gpus'])
                    
                    
#q.stats.tasks_done                
print("all tasks complete!")